In [ ]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [ ]:
sql = """
SELECT
  rx.month,
  rx.pct,
  SUM(CASE
      WHEN cmpa.type = 'AAF' THEN items --calculate AAF items
      ELSE 0 END) AS aaf_items,
  SUM(CASE
      WHEN cmpa.type = 'AAF' THEN actual_cost -- calculate AAF cost
      ELSE 0 END) AS aaf_cost,
  SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN items --calculate all CMPA items
      ELSE 0 END) AS all_items,
  SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN actual_cost -- calculate all CMPA cost
      ELSE 0 END) AS all_cost,
  IEEE_DIVIDE(SUM(CASE
        WHEN cmpa.type = 'AAF' THEN items
        ELSE 0 END), SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN items
      ELSE 0 END)) AS aaf_percent_items,   -- calculate AAF items proportion
  -- calculate AAF items proportion
  IEEE_DIVIDE(SUM(CASE
        WHEN cmpa.type = 'AAF' THEN actual_cost
        ELSE 0 END), SUM(CASE
      WHEN cmpa.type <> 'exclude' THEN actual_cost
      ELSE 0 END)) AS aaf_percent_cost  -- calculate AAF cost proportion
FROM
  hscic.normalised_prescribing_standard AS rx
JOIN
  measures.cmpa_products AS cmpa --join with CPMA products table
ON
  rx.bnf_code=cmpa.bnf_code
WHERE
  rx.bnf_code IN (
  SELECT
    bnf_code
  FROM
    measures.cmpa_products)
GROUP BY
  month,
  pct
ORDER BY
  month
"""
cmpa_df = bq.cached_read(sql, csv_path='cmpa_df.csv', use_cache=True)
cmpa_df['month'] = cmpa_df['month'].astype('datetime64[ns]')